<h1><span style="color:red">Explore Your Dataset with Holoviz</span></h1>

### This sample notebook will walk you through several visualizations implemented in Holoviz, http://holoviz.org/ 
Author: Enrique Sanchez

## 1. Retrieve survey parameters from the URL

In [ ]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

## 2. Import Holoviz modules

In [ ]:
#!pip install panel
#!pip install holoviews
#!pip install bokeh
#!pip install hvplot
#!pip install datashader
    
# Installing libraries
import panel as pn
import holoviews as hv
from holoviews import opts
import hvplot.pandas
import datashader.geo
import pandas as pd
from holoviews.element.tiles import OSM

# Loading extensions
pn.extension()
hv.extension('bokeh')

## 3. Read the survey file in a Python Dataframe, and explore it by rows

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

absolutePath = "../../temp_csvs/"

# read the csv file
file = open(absolutePath + csv_file, encoding="latin-1")
original_df = pd.read_csv(file)

# original_df = original_df.reset_index()

def select_row(row=0):
    return original_df.loc[row:row+2]

# Interactive call to select_row
row_slider = pn.interact(select_row, row=(0, len(original_df)-1), )
row_slider



## 4. Select variables of different types to expore

In [ ]:
# Finds exisitng qualifiers/non-qualifiers in data set
qualifiers, non_qualifiers = ['---', 'None'], []
for column in original_df.columns:
    if '#' in column:
        qualifier = '#' + column.split('#', )[1]
        if qualifier in qualifiers:
            continue
        qualifiers.append(qualifier)  
    else:   
        non_qualifiers.append(column)

        
# Initializes selector widgets
# Includes: Qualifier selector, Variable selector
q_selector = pn.widgets.Select(name='1. Select Qualifier', options=qualifiers)
v_selector = pn.widgets.Select(name='2. Select Variable(s)', options=[])


# Creates link between selectors
#
# Variable selector is dependent on qualifier selector
def var_trigger(target, event):
    if event.new == 'None':
        target.options = original_df.loc[:, non_qualifiers].columns.tolist()
    else:
        is_qualifier = original_df.columns.str.contains(event.new)
        target.options = original_df.loc[:, is_qualifier].columns.tolist()
    return target

q_selector.link(v_selector, callbacks={'value': var_trigger})


# Displays dataframe and slider widget
kept = []
@pn.depends(v_selector.param.value)
def update_df(choice):
    if pd.isnull(choice):
        return;
    
    def display_df(row=0):
        return original_df.loc[row:row+4, kept+[choice]]
    row_slider = pn.interact(display_df, row=range(0, len(original_df)-1, 5))
    
    return row_slider


# Keep button widget
def keep_trigger(event):
    if v_selector.value in kept:
        return
    kept.append(v_selector.value)
        
keep = pn.widgets.Button(name='Keep Variable')
keep.param.watch(keep_trigger, ['clicks'])


# Save button widget
def save_trigger(event):
    global user_df
    user_df = original_df[kept].copy()

save = pn.widgets.Button(name='Save Table')
save.param.watch(save_trigger, ['clicks'])

# Displays widgets produced above
selectors1 = pn.Column(q_selector, v_selector)
buttons = pn.Column(keep, save, margin=(10,0,0,0))
pn.Row(pn.Column(selectors1, buttons), pn.Column(update_df, margin=(0,0,0,20)))

## 5. Select dataset to analyze (original or the one just saved)

In [ ]:
# Determines the existance of a user created data frame
possible_opts = ['---','Original', 'Saved']
saved = 'user_df' in globals()
original = 'original_df' in globals()
true_opts = possible_opts[:original+saved+1]

# Uses user input to choose data frame 
def df_selected(event):
    global df
    if event.new == '---':
        return
    elif event.new == 'Saved':
        df=user_df.copy()
    elif event.new == 'Original':
        df=original_df.copy()

df_selection = pn.widgets.Select(name='Select table to analyze:', options=true_opts)
df_selection.param.watch(df_selected, ['value'])

df_selection

## 6. Statistical summaries for variables and subsets

In [ ]:
# Finding quantitative and qualitative variables
#
# Determines types of values to compare variables (this cell)
#
# Determines options available for selectors in visualizations (next cell)
# e.g. quantitative for x & y (scatter), qualitative 
# for identifier
quantitative, qualitative = [], []
for column in df.columns:
    if df[column].dtype == 'O':
        qualitative.append(column)
        continue
    quantitative.append(column)

# Initializes selector widgets
# Includes: Column selector, Comparison selector, Value selector, 
#           Value slider, Info selector
comp_operators = ['None', 'less than', 'greater than', 'equal to', 'not equal to']
comparison_selector = pn.widgets.Select(name='Comparison Operator', options=comp_operators)
column_selector = pn.widgets.Select(name='Variable', options=['Entire table']+df.columns.tolist())
value_selector = pn.widgets.Select(name='Value')
value_slider = pn.widgets.FloatSlider(name='Value')
info_select = pn.widgets.Select(name='Variable Statistics', options=df.columns.tolist())


# Displays unique values for variable selected for the
# Column selector through a widget. This is either
# a slider (quantitative) or a selector (qualitative)
@pn.depends(column_selector.param.value)
def show_values(column):
    if column == 'Entire table':
        return pn.Row(value_selector, width=150)
    
    elif column in quantitative:
        value_slider.start = df[column].min()
        value_slider.end = df[column].max()
        comparison_selector.options = comp_operators
        return pn.Row(value_slider, width=150)
    
    else:
        options = df[column].unique().tolist()
        value_selector.options = ['None']+options
        comparison_selector.options = ['None', 'equal to', 'not equal to']
        return pn.Row(value_selector, width=150)

    
# Helper function – Filters data based on the
# user's selected options.
def filter_data(col, comp, val_select, val_slide):
    if col == 'Entire table':
        return df
    
    elif col in quantitative:
        if comp == 'less than':
            filtered = df[df[col] < val_slide]
        elif comp == 'greater than':
            filtered = df[df[col] > val_slide]
        elif comp == 'equal to':
            filtered = df[df[col] == val_slide]
        else:
            filtered = df[df[col] != val_slide]
        
    else:
        if comp == 'not equal to':
            filtered = df[df[col] != val_select]
        else:
            filtered = df[df[col] == val_select]
            
    return filtered

# Main function – Displays filtered data frame and column statistics.
@pn.depends(column_selector.param.value, comparison_selector.param.value, 
            value_selector.param.value, value_slider.param.value, info_select.param.value)
def display_data(col, comp, val_select, val_slide, info_col):
    
    # User selects 'Entire table'
    if col == 'Entire table':
        comparison_selector.disabled = True
        value_selector.disabled = True
        
        # Data frame + slider
        def select_row2(row=0):
            return df.iloc[row:row+3]
        row_slider2 = pn.interact(select_row2, row=(0, len(df)-1))
        up_row = pn.Row(row_slider2, margin=(-20,0,0,0))
        
        # Column info
        info = df[[info_col]].describe().T.reset_index(drop=True)
        info_widget = pn.Row(info, margin=(-10,0,0,480))
        
        return pn.Column(info_widget, up_row)
    
    # Ensures comparison and value are enabled
    elif comparison_selector.disabled:
        comparison_selector.disabled = False
        value_selector.disabled = False
    
    # Data frame not displayed if expression not complete
    if (comp == 'None') or (val_select == 'None'):
        return
    
    # Filters dataframe and and creates information table
    filtered = filter_data(col, comp, val_select, val_slide)
    info = filtered[[info_col]].describe().T.reset_index(drop=True)
    info_widget = pn.Row(info, margin=(-10,0,0,480))
    
    # Row slider will not function correctly if data frame is of size 1.
    if len(filtered) <= 1:
        return pn.Column(info_widget, filtered)
    else:
        def select_row3(row=0):
            return filtered.iloc[row:row+3]
        row_slider3 = pn.interact(select_row3, row=(0, len(filtered)-1))
        up_row = pn.Row(row_slider3, margin=(-20,0,0,0))

        return pn.Column(info_widget, up_row)

# Displays widgets produced above
var_comp = pn.Row(column_selector, comparison_selector, width=300, height=80)
expression = pn.Row(var_comp, show_values, css_classes=['widget-box'])
info = pn.Column(info_select, margin=(0,0,0,20))
head = pn.Row(expression, info)
pn.Column(head, display_data)

## 7. Visualize the dataset

In [ ]:
# Finding qualitative variables that contain non-unique values
#
# Prevents groupby selector from displaying all values
# in a column containing only unique values
unique = []
for col in qualitative:
    col_data = df[col].dropna()
    size = len(col_data)
    n_unique = col_data.nunique()
    if n_unique >= 100:
        continue
    if col_data.nunique() < size-1:
        unique.append(col)


# Detects latitude and longitude columns and
# creates data frame containing these coordinates.
has_coords = False
lat, lon = [],[] 

for col in df.columns:
    if '#number#hidden' in col:
        if 'lon' in col:
            lon.append(col)
        elif 'lat' in col:
            lat.append(col)
    elif 'latitude' in col.lower():
        lat.append(col)
    elif 'longitude' in col.lower():
        lon.append(col)

if len(lat) > 0 and len(lon) > 0:
    has_coords = True
    x, y = datashader.geo.lnglat_to_meters(df[lon[0]], df[lat[0]])
    coordinates = df.join([pd.DataFrame({'easting': x}), 
                           pd.DataFrame({'northing': y})])

        
# Defining available plot types – for user
uni = ['histogram', 'boxplot']
multi = ['scatter']
group = ['density']
maps = []
if has_coords:
    maps = ['map']


# Initializes selector widgets
# Includes: Plot type, X variable, Y variable, 
#           Identifer, Size, Groupby, Subgroup
p_selector = pn.widgets.Select(name='1. Plot Type', options=multi+uni+group+maps)
x_selector = pn.widgets.Select(name='2. X Variable', options=quantitative)
y_selector = pn.widgets.Select(name='3. Y Variable', options=quantitative)
identifier = pn.widgets.Select(name='Identifier', options=qualitative)
size = pn.widgets.FloatSlider(name='Size', start=3, value=6, end=12)
g_selector = pn.widgets.Select(name='Groupby (Density Only)', options=['None']+unique)
sg_selector = pn.widgets.Select(name='Subgroup', options=['None'])
dummy = pn.widgets.Select(name='Dummy')


# Helper function – Disables/enables widgets
def disabler(x, y, s, i, g, sg):
    x_selector.disabled = x
    y_selector.disabled = y
    size.disabled = s
    identifier.disabled = i
    g_selector.disabled = g
    sg_selector.disabled = sg
    
    
# Master plotting function
@pn.depends(p_selector.param.value, x_selector.param.value, y_selector.param.value,
            identifier.param.value, size.param.value, g_selector.param.value, 
            sg_selector.param.value)
def plotter(p_selector, x, y, identifier, size, group_col, sg_value):
    
    # Defining available plot types – for hvplot()
    multi = ['scatter']
    uni = ['histogram', 'boxplot']
    
    if (x == y) and not (p_selector in uni+group+maps):
        disabler(False, False, False, False, True, True)
        return
    
    # Multivariate plots
    elif p_selector in multi:
        disabler(False, False, False, False, True, True)
        ident = []
        if len(qualitative) > 0:
            ident = [identifier]
        
        # Scatter plots with more than 4000 points significantly increase lag in plot
        # interactivity. HoloViz's datashade made to alleviate these situations.
        if len(df) > 4000:
            plot = df.hvplot(x, y, hover_cols=ident, datashade=True,
                         hover_color='red', kind=p_selector).opts(frame_height=300)
        else:
            plot = df.hvplot(x, y, hover_cols=ident, hover_color='red', 
                             kind=p_selector).opts(frame_height=300, size=size)
    
    # Univariate plots
    elif p_selector in uni:
        disabler(False, True, True, True, True, True)
        choice = [i for i in ['hist', 'box'] if p_selector.startswith(i)][0]
        horizontal = False
        
        if choice == 'box':
            horizontal = True
            
        plot = df.hvplot(y=x, hover_color='red', kind=choice, invert=horizontal).opts(frame_height=300)
    
    # Density/groupby plot
    elif p_selector in group:
        disabler(False, True, True, True, False, False)
        
        if (group_col != 'None') & (sg_value != 'None'):   
            filtered_df = df[df[group_col] == sg_value]
            plot = filtered_df.hvplot(y=x, kind='kde').opts(frame_height=300)
        else:
            plot = df.hvplot(y=x, kind='kde').opts(frame_height=300)

    # Map plot
    elif p_selector in maps:
        disabler(True, True, False, False, True, True)
        ident = []
        if len(qualitative) > 0:
            ident = [identifier]
        plot = OSM() * coordinates.hvplot.points(x='easting', y='northing',
                                                 hover_cols=ident, size=size)
        
    return plot


# Produces options for 'Subgroup' after selecting groupby
def group_trigger(event):
    if g_selector.value == 'None':
        sg_selector.value == 'None'
        sg_selector.disabled = True
        return
    sg_selector.disabled = False
    sg_selector.options = ['None'] + df[g_selector.value].dropna().unique().tolist()
    
g_selector.param.watch(group_trigger, ['value'])


# Displays widgets produced above
selectors2 = pn.Row(p_selector, x_selector, y_selector)
scatter_options = pn.Column(identifier, size, margin=(40,0,0,0), 
                            width=250, css_classes=['widget-box'])

group_options = pn.Column(g_selector, sg_selector, margin=(20,0,0,0), 
                          width=250, css_classes=['widget-box'])

toolbar = pn.Column(scatter_options, group_options, margin=(0,10,0,0))

pn.Column(selectors2, pn.Row(toolbar, plotter))